In [77]:
#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score

import numpy as np

import keras

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [4]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [5]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [6]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [7]:
#converting GDP quarterly data into monthly

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [8]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)

In [9]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [10]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
dataset = dataset.iloc[552:]
dataset = dataset.iloc[:-59]

In [11]:
y = dataset['Recession'] #splitting off Y

In [12]:
dataset = dataset.drop(columns = ['GDP','Recession']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [13]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    dataclean.clean_zeros(col, dataset)

In [14]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [15]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [16]:
X = dataset

In [17]:
### Data Prep Finished Here ###

In [18]:
#implementing RNN

In [101]:
window = -12
y_shift = y.shift(window) #shifting y to forecast 3 months out
y_shift = y_shift.fillna(0)
#time_step = window*-1

In [102]:
def windowize_data_x(data, n_prev):
    n_predictions = len(data) - n_prev
    y = data[n_prev:]
    # this might be too clever
    indices = np.arange(n_prev) + np.arange(n_predictions)[:, None]
    x = data[indices, None]
    return x

In [103]:
def split_and_windowize_x(data, n_prev, fraction_test=0.3):
    n_predictions = len(data) - 2*n_prev
    
    n_test  = int(fraction_test * n_predictions)
    n_train = n_predictions - n_test   
    
    x_train, y_train = windowize_data(data[:n_train], n_prev)
    x_test, y_test = windowize_data(data[n_train:], n_prev)
    return x_train, x_test

In [129]:
y

1959-01    0.0
1959-02    0.0
1959-03    0.0
1959-04    0.0
1959-05    0.0
1959-06    0.0
1959-07    0.0
1959-08    0.0
1959-09    0.0
1959-10    0.0
1959-11    0.0
1959-12    0.0
1960-01    0.0
1960-02    0.0
1960-03    0.0
1960-04    1.0
1960-05    1.0
1960-06    1.0
1960-07    0.0
1960-08    0.0
1960-09    0.0
1960-10    1.0
1960-11    1.0
1960-12    1.0
1961-01    0.0
1961-02    0.0
1961-03    0.0
1961-04    0.0
1961-05    0.0
1961-06    0.0
          ... 
2012-10    0.0
2012-11    0.0
2012-12    0.0
2013-01    0.0
2013-02    0.0
2013-03    0.0
2013-04    0.0
2013-05    0.0
2013-06    0.0
2013-07    0.0
2013-08    0.0
2013-09    0.0
2013-10    0.0
2013-11    0.0
2013-12    0.0
2014-01    0.0
2014-02    0.0
2014-03    0.0
2014-04    0.0
2014-05    0.0
2014-06    0.0
2014-07    0.0
2014-08    0.0
2014-09    0.0
2014-10    0.0
2014-11    0.0
2014-12    0.0
2015-01    0.0
2015-02    0.0
2015-03    0.0
Name: Recession, Length: 675, dtype: float64

In [148]:
window = 10
window_df = y
columns = ['T']
for x in range(1,window):
    timestring = "T-{}".format(x)
    
    window_df = pd.concat([window_df,y.shift(x*-1)],axis=1)
    #window_df = window_df.rename(columns={window_df.columns[x-1]: timestring })
    columns.append(timestring)
    
window_df.columns = columns

In [149]:
window_df

,T,T-1,T-2,T-3,T-4,T-5,T-6,T-7,T-8,T-9
1959-01,0,0,0,0,0,0,0,0,0,0
1959-02,0,0,0,0,0,0,0,0,0,0
1959-03,0,0,0,0,0,0,0,0,0,0
1959-04,0,0,0,0,0,0,0,0,0,0
1959-05,0,0,0,0,0,0,0,0,0,0
1959-06,0,0,0,0,0,0,0,0,0,0
1959-07,0,0,0,0,0,0,0,0,0,1
1959-08,0,0,0,0,0,0,0,0,1,1
1959-09,0,0,0,0,0,0,0,1,1,1
1959-10,0,0,0,0,0,0,1,1,1,0


In [132]:
windowed = pd.concat([y,y.shift(-1)],axis=1)

In [133]:
windowed

,Recession,Recession
1959-01,0.0,0.0
1959-02,0.0,0.0
1959-03,0.0,0.0
1959-04,0.0,0.0
1959-05,0.0,0.0
1959-06,0.0,0.0
1959-07,0.0,0.0
1959-08,0.0,0.0
1959-09,0.0,0.0
1959-10,0.0,0.0


In [ ]:
#making function to windowize y
def window_y(y_data,window):
    

In [104]:
n_prev = 50

x_train, x_test, y_train = split_and_windowize(X, n_prev)
x_train.shape, x_test.shape

NameError: name 'split_and_windowize' is not defined

In [105]:
X_train = X.iloc[12:550]
X_test = X.iloc[550:]
y_train = y_shift.iloc[12:550]
y_test = y_shift.iloc[550:]

In [106]:
X_train.shape

(538, 98)

In [107]:
X_train.shape[1]

98

In [108]:
np.reshape(X_train.values,(538,1,98)) #worked with 1

array([[[61.5       ,  5.2       , 10.9       , ...,  0.        ,
          1.03448276, -0.31545741]],

       [[52.3       ,  4.8       , 10.2       , ...,  0.31746032,
          1.73010381, -0.31545741]],

       [[47.8       ,  5.4       , 11.5       , ...,  0.95238095,
          1.73010381,  0.31545741]],

       ...,

       [[58.5       ,  5.4       , 11.5       , ...,  0.81743869,
          2.65438787,  7.24637681]],

       [[57.4       ,  5.4       , 11.7       , ...,  0.33967391,
          2.53779698,  6.64259928]],

       [[56.3       ,  5.5       , 12.1       , ...,  1.76390773,
          3.18918919,  7.68126346]]])

In [109]:
trainX = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1])) #this shouldn't throw an error, X_train is 2D

In [110]:
trainX.shape

(538, 1, 98)

In [111]:
y_train.shape

(538,)

In [155]:
model = keras.Sequential()
model.add(keras.layers.LSTM(64, input_shape=(1,98), return_sequences=True)) #shouldn't this be 538,1,98?
model.add(keras.layers.LSTM(64, return_sequences=True))
model.add(keras.layers.LSTM(64, return_sequences=True))
model.add(keras.layers.LSTM(64, return_sequences=True))
model.add(keras.layers.LSTM(64, return_sequences=True))
model.add(keras.layers.LSTM(64, return_sequences=False))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy') #this is log loss

In [156]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 1, 64)             41728     
_________________________________________________________________
lstm_29 (LSTM)               (None, 1, 64)             33024     
_________________________________________________________________
lstm_30 (LSTM)               (None, 1, 64)             33024     
_________________________________________________________________
lstm_31 (LSTM)               (None, 1, 64)             33024     
_________________________________________________________________
lstm_32 (LSTM)               (None, 1, 64)             33024     
_________________________________________________________________
lstm_33 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 65        
Total para

In [167]:
model.fit(trainX, y_train.values, batch_size=64, epochs=1000)

Epoch 1/1000
538/538 [==============================] - 0s 592us/step - loss: 0.0583
Epoch 2/1000
538/538 [==============================] - 0s 604us/step - loss: 0.0573
Epoch 3/1000
538/538 [==============================] - 0s 688us/step - loss: 0.0560
Epoch 4/1000
538/538 [==============================] - 0s 711us/step - loss: 0.0547
Epoch 5/1000
538/538 [==============================] - 0s 801us/step - loss: 0.0570
Epoch 6/1000
538/538 [==============================] - 0s 698us/step - loss: 0.0555
Epoch 7/1000
538/538 [==============================] - 0s 680us/step - loss: 0.0556
Epoch 8/1000
538/538 [==============================] - 0s 655us/step - loss: 0.0566
Epoch 9/1000
538/538 [==============================] - 0s 649us/step - loss: 0.0548
Epoch 10/1000
538/538 [==============================] - 0s 631us/step - loss: 0.0565
Epoch 11/1000
538/538 [==============================] - 0s 687us/step - loss: 0.0572
Epoch 12/1000
538/538 [==============================] - 0s 642

538/538 [==============================] - 0s 698us/step - loss: 0.0557
Epoch 97/1000
538/538 [==============================] - 0s 894us/step - loss: 0.0559
Epoch 98/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0540
Epoch 99/1000
538/538 [==============================] - 1s 967us/step - loss: 0.0545
Epoch 100/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0563
Epoch 101/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0565
Epoch 102/1000
538/538 [==============================] - 1s 984us/step - loss: 0.0551
Epoch 103/1000
538/538 [==============================] - 0s 848us/step - loss: 0.0554
Epoch 104/1000
538/538 [==============================] - 0s 785us/step - loss: 0.0565
Epoch 105/1000
538/538 [==============================] - 1s 995us/step - loss: 0.0556
Epoch 106/1000
538/538 [==============================] - 0s 882us/step - loss: 0.0547
Epoch 107/1000
538/538 [==============================] - 1s 1ms/st

538/538 [==============================] - 1s 951us/step - loss: 0.0555
Epoch 192/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0552
Epoch 193/1000
538/538 [==============================] - 0s 923us/step - loss: 0.0567
Epoch 194/1000
538/538 [==============================] - 0s 873us/step - loss: 0.0534
Epoch 195/1000
538/538 [==============================] - 0s 853us/step - loss: 0.0560
Epoch 196/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0536
Epoch 197/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0562
Epoch 198/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0547
Epoch 199/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0574
Epoch 200/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0538
Epoch 201/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0574
Epoch 202/1000
538/538 [==============================] - 1s 2ms/step - 

538/538 [==============================] - 1s 1ms/step - loss: 0.0543
Epoch 288/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0539
Epoch 289/1000
538/538 [==============================] - 0s 787us/step - loss: 0.0529
Epoch 290/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0550
Epoch 291/1000
538/538 [==============================] - 0s 868us/step - loss: 0.0521
Epoch 292/1000
538/538 [==============================] - 0s 856us/step - loss: 0.0550
Epoch 293/1000
538/538 [==============================] - 0s 755us/step - loss: 0.0533
Epoch 294/1000
538/538 [==============================] - 0s 787us/step - loss: 0.0530
Epoch 295/1000
538/538 [==============================] - 0s 816us/step - loss: 0.0542
Epoch 296/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0531
Epoch 297/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0523
Epoch 298/1000
538/538 [==============================] - 1s 1ms/ste

538/538 [==============================] - 0s 754us/step - loss: 0.0518
Epoch 382/1000
538/538 [==============================] - 0s 786us/step - loss: 0.0541
Epoch 383/1000
538/538 [==============================] - 0s 833us/step - loss: 0.0519
Epoch 384/1000
538/538 [==============================] - 0s 853us/step - loss: 0.0538
Epoch 385/1000
538/538 [==============================] - 0s 847us/step - loss: 0.0511
Epoch 386/1000
538/538 [==============================] - 0s 876us/step - loss: 0.0521
Epoch 387/1000
538/538 [==============================] - 0s 815us/step - loss: 0.0513
Epoch 388/1000
538/538 [==============================] - 0s 825us/step - loss: 0.0537
Epoch 389/1000
538/538 [==============================] - 0s 805us/step - loss: 0.0521
Epoch 390/1000
538/538 [==============================] - 0s 855us/step - loss: 0.0519
Epoch 391/1000
538/538 [==============================] - 0s 836us/step - loss: 0.0532
Epoch 392/1000
538/538 [==============================] - 

...

Epoch 571/1000
538/538 [==============================] - 0s 745us/step - loss: 0.0518
Epoch 572/1000
538/538 [==============================] - 0s 716us/step - loss: 0.0503
Epoch 573/1000
538/538 [==============================] - 0s 735us/step - loss: 0.0499
Epoch 574/1000
538/538 [==============================] - 0s 736us/step - loss: 0.0528
Epoch 575/1000
538/538 [==============================] - 0s 745us/step - loss: 0.0487
Epoch 576/1000
538/538 [==============================] - 0s 739us/step - loss: 0.0503
Epoch 577/1000
538/538 [==============================] - 0s 709us/step - loss: 0.0506
Epoch 578/1000
538/538 [==============================] - 0s 715us/step - loss: 0.0514
Epoch 579/1000
538/538 [==============================] - 0s 710us/step - loss: 0.0516
Epoch 580/1000
538/538 [==============================] - 0s 732us/step - loss: 0.0504
Epoch 581/1000
538/538 [==============================] - 0s 711us/step - loss: 0.0499
Epoch 582/1000
538/538 [===================

538/538 [==============================] - 1s 1ms/step - loss: 0.0528
Epoch 666/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0495
Epoch 667/1000
538/538 [==============================] - 0s 927us/step - loss: 0.0505
Epoch 668/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0500
Epoch 669/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0539
Epoch 670/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0500
Epoch 671/1000
538/538 [==============================] - 1s 941us/step - loss: 0.0503
Epoch 672/1000
538/538 [==============================] - 0s 895us/step - loss: 0.0503
Epoch 673/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0501
Epoch 674/1000
538/538 [==============================] - 0s 850us/step - loss: 0.0493
Epoch 675/1000
538/538 [==============================] - 1s 1ms/step - loss: 0.0511
Epoch 676/1000
538/538 [==============================] - 1s 1ms/step - 

538/538 [==============================] - 0s 690us/step - loss: 0.0492
Epoch 760/1000
538/538 [==============================] - 0s 727us/step - loss: 0.0508
Epoch 761/1000
538/538 [==============================] - 0s 689us/step - loss: 0.0497
Epoch 762/1000
538/538 [==============================] - 0s 694us/step - loss: 0.0494
Epoch 763/1000
538/538 [==============================] - 0s 667us/step - loss: 0.0494
Epoch 764/1000
538/538 [==============================] - 0s 698us/step - loss: 0.0496
Epoch 765/1000
538/538 [==============================] - 0s 683us/step - loss: 0.0497
Epoch 766/1000
538/538 [==============================] - 0s 679us/step - loss: 0.0518
Epoch 767/1000
538/538 [==============================] - 0s 671us/step - loss: 0.0506
Epoch 768/1000
538/538 [==============================] - 0s 679us/step - loss: 0.0494
Epoch 769/1000
538/538 [==============================] - 0s 706us/step - loss: 0.0499
Epoch 770/1000
538/538 [==============================] - 

538/538 [==============================] - 0s 713us/step - loss: 0.0483
Epoch 854/1000
538/538 [==============================] - 0s 662us/step - loss: 0.0511
Epoch 855/1000
538/538 [==============================] - 0s 668us/step - loss: 0.0495
Epoch 856/1000
538/538 [==============================] - 0s 663us/step - loss: 0.0507
Epoch 857/1000
538/538 [==============================] - 0s 713us/step - loss: 0.0510
Epoch 858/1000
538/538 [==============================] - 0s 668us/step - loss: 0.0497
Epoch 859/1000
538/538 [==============================] - 0s 717us/step - loss: 0.0488
Epoch 860/1000
538/538 [==============================] - 0s 720us/step - loss: 0.0502
Epoch 861/1000
538/538 [==============================] - 0s 692us/step - loss: 0.0497
Epoch 862/1000
538/538 [==============================] - 0s 673us/step - loss: 0.0508
Epoch 863/1000
538/538 [==============================] - 0s 694us/step - loss: 0.0505
Epoch 864/1000
538/538 [==============================] - 

538/538 [==============================] - 0s 836us/step - loss: 0.0498
Epoch 948/1000
538/538 [==============================] - 0s 744us/step - loss: 0.0505
Epoch 949/1000
538/538 [==============================] - 0s 784us/step - loss: 0.0499
Epoch 950/1000
538/538 [==============================] - 0s 745us/step - loss: 0.0497
Epoch 951/1000
538/538 [==============================] - 0s 756us/step - loss: 0.0494
Epoch 952/1000
538/538 [==============================] - 0s 730us/step - loss: 0.0492
Epoch 953/1000
538/538 [==============================] - 0s 741us/step - loss: 0.0509
Epoch 954/1000
538/538 [==============================] - 0s 786us/step - loss: 0.0496
Epoch 955/1000
538/538 [==============================] - 1s 2ms/step - loss: 0.0502
Epoch 956/1000
538/538 [==============================] - 0s 926us/step - loss: 0.0500
Epoch 957/1000
538/538 [==============================] - 0s 820us/step - loss: 0.0501
Epoch 958/1000
538/538 [==============================] - 0s

In [168]:
testX = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1])) #this shouldn't throw an error, X_train is 2D

In [169]:
y_pred = model.predict(testX)

In [170]:
log_loss(y_test,y_pred)

0.3841698722734509

In [171]:
roc_auc_score(y_test,y_pred)

0.7750737463126843